In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import sys
sys.path.append('..')
sys.path.append('../../..')

from hamiltonian_monte_carlo import hmc
from hamiltonian_monte_carlo.nuts import NoUTurnSampler
from distributions import BivariateGaussian, BivariateBanana

In [ ]:
def plot_2d_empirical_hist(samples, bins, marginal_pdf):

    plt.subplot(2, 2, 1)
    plt.hist2d(samples[0, :], samples[1, :],
               bins=bins, density=True, cmap='inferno')
    plt.ylabel(r'$\theta_2$')
    plt.xlabel(r'$\theta_1$')
    plt.colorbar(ticks=[])

    for axis in [0, 1]:
        plt.subplot(2, 2, 3 - axis)
        plt.hist(samples[axis, :], bins=bins[axis], density=True)
        plt.plot(bins[axis], marginal_pdf[axis])
        plt.xlabel(r'$\theta_{:d}$'.format(axis + 1))
    
    plt.tight_layout()

## Run HMC on a bivariate Gaussian target.

In [ ]:
bivariate_gauss = BivariateGaussian(rho=.9, sigma=np.array([1., 2.]))
pc_var, _ = bivariate_gauss.get_principal_components()
pc_scale = np.sqrt(pc_var)

# Choose near-optimal stepsize and integration time.
dt = np.array([.7, .8]) * 2. * np.min(pc_scale)
n_step = np.ceil(
    np.array([.9, 1.1]) * np.pi / 2 * np.max(pc_scale) / np.max(dt)
)

n_burnin = 10 ** 4
n_sample = 10 ** 5

theta0 = np.zeros(2)
f = bivariate_gauss.compute_logp_and_gradient
samples, logp_samples, accept_prob, time_elapsed \
    = hmc.generate_samples(f, theta0, n_burnin, n_sample, n_step)

In [ ]:
plt.figure(figsize=(14, 10))
plt.rcParams['font.size'] = 20

bins = [
    bivariate_gauss.sigma[0] * np.linspace(-4, 4, 51),
    bivariate_gauss.sigma[1] * np.linspace(-4, 4, 51)
]
marginal_pdf = bivariate_gauss.compute_marginal_pdf(*bins)

plot_2d_empirical_hist(samples, bins, marginal_pdf)
plt.show()

## Run HMC on a 2-d banana-shaped distribution.
Gaussians have many special features that makes them particularly simple for HMC, so additional sanity check with a non-Gaussian target won't hurt.

In [ ]:
banana = BivariateBanana()
dt = .2
n_step = [7, 9]

n_burnin = 10 ** 3
n_sample = 10 ** 5

theta0 = np.array([1., 0.])
f = banana.compute_logp_and_gradient
samples, info \
    = hmc.generate_samples(f, theta0, n_burnin, n_sample, n_step)

In [ ]:
plt.figure(figsize=(14, 10))
plt.rcParams['font.size'] = 20

bins = 2 * [np.linspace(-3, 3, 51)]
marginal_pdf = banana.compute_marginal_pdf(*bins)

plot_2d_empirical_hist(samples, bins, marginal_pdf)
plt.show()

## Run NUTS on a 2-d banana-shaped distribution.

In [ ]:
banana = BivariateBanana()

n_burnin = 10 ** 4
n_sample = 10 ** 5

theta0 = np.array([1., 0.])
f = banana.compute_logp_and_gradient
mass = np.array([4., 1.])
nuts = NoUTurnSampler(f, mass)
samples, info \
    = nuts.generate_samples(theta0, n_burnin, n_sample, n_update=10, seed=0)

In [ ]:
plt.figure(figsize=(14, 10))
plt.rcParams['font.size'] = 20

bins = 2 * [np.linspace(-3, 3, 51)]
marginal_pdf = banana.compute_marginal_pdf(*bins)

plot_2d_empirical_hist(samples, bins, marginal_pdf)
plt.show()

In [ ]:
plt.plot(info['stepsize'][:n_burnin])
plt.show()